# extract text and its position

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO


def convert_pdf_to_txt(path1, path2, num):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path1, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        interpreter.process_page(page)
        if i >= num:
            break
    text = retstr.getvalue()
    file_name = 'page' + str(num)
    f = open(path2+file_name, 'w')
    f.write(text)
    f.close()
    #text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()



In [2]:
path1 = '/home/zhongzhu/Desktop/Project2/Test/Seattle.pdf'
path2 = '/home/zhongzhu/Desktop/Project2/Test/temp/'
convert_pdf_to_txt(path1, path2, 200)

# record positions

In [3]:
import re

path1 = '/home/zhongzhu/Desktop/Project2/Test/Seattle.pdf'
path2 = '/home/zhongzhu/Desktop/Project2/Test/temp/'

def record_pos(path, page, LINE_HEIGHT = 1):
    f = open(path2+'page'+str(page), 'r')
    boxes = f.read().split('TEXT_BOX')
    f.close()

    contents = {}
    for content in boxes:
        c = content.split(';')
        if(c[0].find('left:') > 0): #if not empty
            l = c[0][c[0].find('left:')+5:c[0].find('px')]
            t = c[1][c[1].find('top:')+4:c[1].find('px')]
            w = c[2][c[2].find('width:')+6:c[2].find('px')]
            h = c[3][c[3].find('height:')+7:c[3].find('px')]

            text = "".join(c[4:])
            lines = text.split('\n')
                
            tempCount = 0
            for line in lines:
                if re.search(r'[A-Za-z0-9,.?!]', line):
                    if not line.decode('utf-8') in contents:
                        contents[line.decode('utf-8')] = [(int(l), int(t)+tempCount, int(w), int(h))]
                    else:
                        contents[line.decode('utf-8')].append((int(l), int(t)+tempCount, int(w), int(h)))
                tempCount += LINE_HEIGHT ### default value 1
    return contents


In [4]:
contents = record_pos(path2, 200)
for pos in contents:
    print(pos)
    print(contents[pos])

200' total sew er installed
[(95, 60078, 401, 35)]
Total operating hours to date: 2,536.0
[(95, 81321, 126, 189)]
Total operating hours to date: 232.5
[(95, 23122, 120, 98)]
Excavator
[(95, 107381, 181, 169)]
Total operating hours to date: 604.5
[(95, 38138, 120, 47)]
Received missing draw ings of 65% storm drain south outfall - given to COE
[(95, 80488, 256, 18)]
Cat320
[(95, 40064, 156, 98), (197, 41732, 80, 165), (197, 45109, 80, 145), (197, 54359, 80, 145), (197, 56007, 61, 85), (95, 57722, 163, 149), (95, 59389, 163, 159), (95, 61066, 163, 179), (95, 71179, 163, 159), (197, 73470, 70, 145), (197, 75377, 99, 145), (197, 77679, 99, 175), (197, 81314, 99, 175), (197, 89661, 85, 156), (197, 94830, 85, 135), (95, 96328, 163, 78), (197, 99726, 80, 135), (95, 101607, 181, 159)]
2&quot Conduit ufer - grd ¾ grd, conduit
[(95, 154559, 120, 18)]
Cut H2O main on road R back fill w ith coader
[(95, 124010, 151, 18)]
P51415
[(95, 15705, 36, 20), (95, 77018, 36, 61)]
Install 12&quot storm drain 

In [5]:
filted_contents = {}
for text in contents:
    if len(contents[text]) > 60:  ###set at beginning#####################
        filted_contents[text] = contents[text]
        print(text + ' ' +str(len(filted_contents[text])))

 LABOR HOURS 92
 REPORT NUMBER 201
 Standby 84
6.0 74
10.0 183
10.5 68
 CONTRACTORS ON SITE (Report first and/or last day contractors w ere on site) 91
 Number of 90
 QUALITY CONTROL INSPECTIONS AND RESULTS 87
LABORERS 140
 SUPERINTENDENT'S INITIALS 92
 PATTERSON 73
 CONTRACTORS FIRST AND/OR LAST DAY ONSITE 88
 The following equipment hours were Reported today: 84
 No accidents reported today 92
Asbestos Abatement 118
material used and work performed during this Reporting period are in compliance with the contract 92
 PREP/INITIAL DATES 92
 Labor Classification 89
 Description 140
 CONTRACTORS QUALITY CONTROL REPORT (QCR) 201
 QC REQUIREMENTS 92
 0.0 89
 ACCIDENT REPORTING 92
 Total 174
 REMARKS 91
Hours 168
 PREPARATORY AND/OR INITIALS HELD TODAY 92
 CONTRACT NUMBER 201
plans and specifications, to the best of my knowledge, except as noted above. 92
544H Loader 66
 Hours 90
 The following labor hours were Reported today: 90
 (Preparatory and initial dates held and advance notice) 92
 

In [6]:
convert_pdf_to_txt(path1, path2, 1)

# get the positions of the first page
first_page_contents = record_pos(path2, 1)

# search neighbors

In [ ]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(filted_contents, first_page_contents):
    if(line1 != line2 and line1 in first_page_contents
       and abs(int(first_page_contents[line1][0][0])-int(first_page_contents[line2][0][0]))<1000    ###set at beginning
       and abs(int(first_page_contents[line1][0][1])-int(first_page_contents[line2][0][1]))<100): ###set at beginning
        key = line1
        value = line2
        if not key in neighbors:
            neighbors[key] = []
        neighbors[key].append(value)

# GUI

In [ ]:
import Tkinter as tk
import ttk

win = tk.Tk()
win.title("Set Pairs")   

ttk.Label(win, text="Select category:").grid(column=0, row=0)   
ttk.Label(win, text="content start from").grid(column=3, row=0)  
ttk.Label(win, text="next category").grid(column=3, row=0)      


def select():
    dropdown2['values'] = neighbors[category.get()]  
    dropdown2.current(0)  
    dropdown3['values'] = neighbors[category.get()]  
    dropdown3.current(0)    
    


pairs = {}
# 
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    
    key = category.get()
    a = contents[category.get()][0]
    b = contents[content.get()][0]
    new_value = (b[0]-a[0], b[1]-a[1], b[2]-a[2], b[3]-a[3])
    end = next_cat.get()
    
    if not key in pairs:
        pairs[key] = set([])
    pairs[key].add((new_value, end))
    dropdown2.current(0) 
    
# creat a drop-down list
category = tk.StringVar()
dropdown1 = ttk.Combobox(win, width=40, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())     
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0)     

# selecte button
action = ttk.Button(win, text="Select", command=select)
action.grid(column=1, row=1)   

# create another drop-down list
content = tk.StringVar()
dropdown2 = ttk.Combobox(win, width=40, textvariable=content)    
dropdown2.grid(column=2, row=1)  

# create third drop-down list
next_cat = tk.StringVar()
dropdown3 = ttk.Combobox(win, width=40, textvariable=next_cat)    
dropdown3.grid(column=3, row=1)     

# bind button
action = ttk.Button(win, text="Bind", command=bind)
action.grid(column=4, row=1) 
win.mainloop()      # window

In [9]:
for i in pairs:
    print i
    print type(pairs[i])
    print (pairs[i])
    for i in pairs[i]:
        print len(i)

 Activities in Progress:
<type 'set'>
set([((10, 10, 113, 8), ' General Comments:')])
2


In [10]:
import re

page = 200
f = open(path2+'page'+str(page), 'r')
text = f.read()
text, num = re.subn(r'TEXT_BOX.*?height:[\d]+px;', '',text)
f.close()

In [11]:
print text[:1000]


 REPORT NUMBER

 91
DATE

 Page 1 of 2

 01 Nov 2004 - Monday

 CONTRACT NUMBER

 DACA67-02-D-2012  0004

 WEATHER
Weather Caused No Delay
Temperature Min 21 °F, Max 44 °F; No Precipitation; No Wind

 CONTRACTORS QUALITY CONTROL REPORT (QCR)

 DAILY LOG OF CONSTRUCTION - MILITARY

 PROJECT

 D/B Replace Military Family Housing P5

 CONTRACTOR

 Evergreene Construction, L.C.

 QC NARRATIVES

 Activities in Progress:

 AAI
Worked on shutting down water and removing refrigerators

 General Comments:

 DAILY CONTROL INSPECTIONS AND RESULTS
None today

 QUALITY CONTROL TESTING AND RESULTS
None today

 REMARKS
No remarks today

 Safety Inspection / Safety Meetings:

 DAILY SAFETY INSPECTIONS
None today

 PREPARATORY AND/OR INITIALS HELD TODAY
Held demo prep meeting today with AAI.  See attached minutes of meeting.  Will start next week with demo.  They brought new 
license, number was okay by Jim LaFluer.

 PREP/INITIAL DATES

 (Preparatory and initial dates held and advance notice)

 No 


In [48]:
def find_content_by_area(text, start, end):
    #p = re.compile(r'{%s}.*{%s}' % (start, end))
    content= []
    search = re.findall('%s.*?%s' % (start, end), text, re.DOTALL)
    for i in search:
        i = i.replace(start, '')
        i = i.replace(end, '')
        i = i.replace('\n\n', '')
        content.append([i])
    print len(content)
    return content

In [50]:
data = {}
for cat in pairs:
    for shift in pairs[cat]:
        if len(shift)==2:
            content = find_content_by_area(text, cat, shift[1])
            data[cat] = content

91


In [49]:
def find_content_by_pos(content_map, content_pos):
    
    content = []
    for c in content_map:
        m = 1 #### maximal error allowed#########################
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m:
                content.append(c)
    #print(m)
    return content
    
data = {}
content_map = record_pos(path2, 500)
for cat in pairs:
    #if cat in content_map:  #unnecessary??
    for pos in content_map[cat]:
        content = []
        for shift in pairs[cat]:
            if len(shift)==2:
                content.append(find_content_by_area(text, cat, shift[1]))
            else:
                content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[2]+shift[2], pos[3]+shift[3])
                content.append(find_content_by_pos(content_map, content_pos))
        if cat in data: #unnecessary??
            data[cat].append(content)
        else:
            data[cat] = [content]
           


91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91


In [51]:
for i in data:
    print i
    for j in data[i]:
        print j
        print len(j)
print len(data)

 Activities in Progress:
[' AAI\nWorked on shutting down water and removing refrigerators']
1
[' EVERGREENE MANAGEMENT\nReview ed tree preservation plan w ith COE, Base and Base Arborist.  Some trees are missing and mislabeled.  Generally okay.  \nSeveral additional trees w ere added to the removal due to the condition of the trees - old split trunks, and poor conditions\nShould there be added costs for removal of these trees (No additional \nDid not finish.  NE area remains.  Scheduled to finish on Wednesday at 10:30 am. AAI\nShutting dow n w ater, and removing refrigerators']
1
[' AAI\nShutting dow n w ater and removing refrigerators']
1
[' AAI\nShutting dow n w ater and removing refrigerators']
1
[' AAI\nShutting dow n w ater and removing refrigerators']
1
[' AAI\nDemolished building #4215']
1
[' AAI(cid:31)\nDemolished building 4264 - crunching and hauling demolition debris.  ']
1
[' AAI(cid:31)\nDemolished garages for buildings #4215 and #4212.  Worked on hauling demolition debris

# generate CSV table

In [52]:
import csv
f = open('generated_table_area.csv', 'w')
writer = csv.writer(f)

for i, cat in enumerate(data):
    if i >50:
        break
    writer.writerow([cat] + data[cat])
f.close()

# test

In [1]:
for i in content_map:
    print i
    print content_map[i]

NameError: name 'content_map' is not defined

In [94]:
len(data[' WEATHER'])

1410

In [62]:
contents = record_pos(path2, 14)
for pos in contents:
    print(pos)
    print(contents[pos])

 Employer * * * * * *
(95, 10799, 71, 23)
None today * * * * * * * * * * * * * * * *
(95, 5474, 175, 18)
 PREP/INITIAL DATES * * * * *
(75, 8658, 75, 10)
 The following activity was started today:
(85, 10613, 156, 10)
 No contractors had their first or last day on site today * * * * * * *
(85, 10755, 206, 10)
 WEATHER * * * * * *
(319, 9436, 229, 28)
 QUALITY CONTROL INSPECTIONS AND RESULTS * * *
(95, 7013, 168, 18)
 DACA67-02-D-2012  0004 * * * * * * * * * * * * *
(424, 11105, 89, 10)
DATE * * * * *
(417, 4324, 16, 22)
None today * * * * * * * * * * * * *
(95, 5330, 168, 18)
Worked *
(508, 2721, 26, 20)
0.0 * * * *
(441, 2722, 36, 44)
 Evergreene Construction, L.C. * * * * * * *
(82, 11970, 102, 10)
 LABOR HOURS * * * * * *
(75, 9859, 53, 10)
 LABOR HOURS
(75, 614, 53, 10)
 No accidents reported today * *
(85, 4445, 109, 10)
 CONTRACTORS ON SITE (Report first and/or last day contractors w ere on site) * * * * * *
(75, 10741, 278, 10)
 General Comments:
(85, 267, 78, 10)
 QA/QC PUNCH L